# GML Final Project Classification on bbbp Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install dgl -f https://data.dgl.ai/wheels/repo.html

!pip install dglgo -f https://data.dgl.ai/wheels-test/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.0 MB/s eta 0:00:00
Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 43.8 MB/s eta 0:00:00
  P

In [3]:
%matplotlib inline
import os

os.environ["DGLBACKEND"] = "pytorch"
import dgl
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import dgl.function as fn
import torch.nn.functional as F
import shutil
from torch.utils.data import DataLoader
import cloudpickle
from dgl.nn import GraphConv

#### Set Path

In [4]:
current_dir = "/content/drive/MyDrive/GML Final Project/"
checkpoint_path = current_dir + "save_models/model_checkpoints/" + "checkpoint"
os.makedirs(checkpoint_path, exist_ok=True)

best_model_path = current_dir + "save_models/best_model/"

folder_data_temp = current_dir +"data_temp/"
shutil.rmtree(folder_data_temp, ignore_errors=True)

path_save = current_dir + "Hiv.zip"
shutil.unpack_archive(path_save, folder_data_temp)

#### Custom PyTorch Datasets

In [5]:
""" Classification Dataset """
class DGLDatasetClass(torch.utils.data.Dataset):
    def __init__(self, address):
      #initializes the dataset by loading the preprocessed graphs, labels, masks, and global features from the specified binary file using the dgl.load_graphs() function.
            self.address=address+".bin"
            self.list_graphs, train_labels_masks_globals = dgl.load_graphs(self.address)
      #The number of graphs in the dataset is calculated and stored as num_graphs.
            num_graphs =len(self.list_graphs)
      #The labels, masks, and global features are extracted from the loaded data and reshaped into 2-dimensional tensors, where each row represents a graph.
            self.labels = train_labels_masks_globals["labels"].view(num_graphs,-1)
            self.masks = train_labels_masks_globals["masks"].view(num_graphs,-1)
            self.globals = train_labels_masks_globals["globals"].view(num_graphs,-1)
      #The __len__() method overrides the base class method and returns the length of the dataset, which is the number of graphs.
    def __len__(self):
        return len(self.list_graphs)
      #The __getitem__(self, idx) method overrides the base class method and returns the graph at the specified index (idx), along with its corresponding label, mask, and global features.
    def __getitem__(self, idx):
        return  self.list_graphs[idx], self.labels[idx], self.masks[idx], self.globals[idx]

#### Defining Train, Validation, and Test Set

In [7]:
path_data_temp = folder_data_temp + "scaffold"+"_"+str(0)
train_set = DGLDatasetClass(address=path_data_temp+"_train")
val_set = DGLDatasetClass(address=path_data_temp+"_val")
test_set = DGLDatasetClass(address=path_data_temp+"_test")

print(len(train_set), len(val_set), len(test_set))


32901 4112 4114


#### Data Loader

In [8]:
def collate(batch):
    # batch is a list of tuples (graphs, labels, masks, globals)
    # Concatenate a sequence of graphs

    ### This line extracts the graphs from each tuple in the batch and creates a list of individual graphs.
    graphs = [e[0] for e in batch]
    ### This line uses the dgl.batch() function to combine the individual graphs into a single graph batch g
    g = dgl.batch(graphs)

    # Concatenate a sequence of tensors (labels) along a new dimension

    ### This line extracts the labels from each tuple in the batch and creates a list of individual label tensors.
    labels = [e[1] for e in batch]
    ### This line uses torch.stack() to concatenate the individual label tensors along a new dimension (dimension 0) and create a single tensor labels representing the labels for the entire batch.
    labels = torch.stack(labels, 0)

    # Concatenate a sequence of tensors (masks) along a new dimension

    ### This line extracts the masks from each tuple in the batch and creates a list of individual mask tensors.
    masks = [e[2] for e in batch]
    ### This line uses torch.stack() to concatenate the individual mask tensors along a new dimension (dimension 0) and create a single tensor masks representing the masks for the entire batch.
    masks = torch.stack(masks, 0)

    # Concatenate a sequence of tensors (globals) along a new dimension

    ### This line extracts the global features from each tuple in the batch and creates a list of individual global feature tensors.
    globals = [e[3] for e in batch]
    ### This line uses torch.stack() to concatenate the individual global feature tensors along a new dimension (dimension 0) and create a single tensor globals representing the global features for the entire batch.
    globals = torch.stack(globals, 0)

    return g, labels, masks, globals


def loader(batch_size=64):
    train_dataloader = DataLoader(train_set,
                              batch_size=batch_size,
                              collate_fn=collate,
                              drop_last=False,
                              shuffle=True,
                              num_workers=1)

    val_dataloader =  DataLoader(val_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)

    test_dataloader = DataLoader(test_set,
                             batch_size=batch_size,
                             collate_fn=collate,
                             drop_last=False,
                             shuffle=False,
                             num_workers=1)
    return train_dataloader, val_dataloader, test_dataloader

In [9]:
train_dataloader, val_dataloader, test_dataloader = loader(batch_size=64)

#### Defining A GNN

##### Some Variables

In [10]:
#Bace dataset has 1 task. Some other datasets may have some more number of tasks, e.g., tox21 has 12 tasks.
num_tasks = 1

# Size of global feature of each graph
global_size = 200

# Number of epochs to train the model
num_epochs = 100

# Number of steps to wait if the model performance on the validation set does not improve
patience = 10

#Configurations to instantiate the model
config = {"node_feature_size":127, "edge_feature_size":12, "hidden_size":100}


In [11]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv3 = GraphConv(self.hidden_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv4 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        h = mol_dgl_graph.ndata["v"]
        h = self.conv1(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv4(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")


#### Function to Compute Score of the Model

In [12]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

#### Loss Function

In [13]:
def loss_func(output, label, mask, num_tasks):
    #This tensor is used as a weight for positive examples in the binary cross-entropy loss.
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    #reduction='none': This specifies that the loss should not be reduced and returned element-wise.
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    #The mask tensor is used to mask out the loss for invalid labels.
    loss = mask*criterion(output,label)
    #The masked loss is summed and divided by the sum of the mask to compute the average loss.
    loss = loss.sum() / mask.sum()
    return loss

#### Training and Evaluation

##### Training Function

In [14]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        # Zeros out the gradients of the model parameters using
        optimizer.zero_grad(set_to_none=True)
        # Computes the gradients of the loss with respect to the model parameters using
        loss_train.backward()
        # Updates the model parameters using
        optimizer.step()
        # Adds the loss for the current batch to
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [15]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


##### Function to compute test set score of the final saved model

In [16]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


##### Train the model and evaluate its performance

In [17]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.229 | Valid Score: 0.557
 
Epoch: 1/100 | Best Valid Score Until Now: 0.557 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.155 | Valid Score: 0.661
 
Epoch: 2/100 | Best Valid Score Until Now: 0.661 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.149 | Valid Score: 0.727
 
Epoch: 3/100 | Best Valid Score Until Now: 0.727 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.145 | Valid Score: 0.742
 
Epoch: 4/100 | Best Valid Score Until Now: 0.742 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.143 | Valid Score: 0.745
 
Epoch: 5/100 | Best Valid Score Until Now: 0.745 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.143 | Valid Score: 0.747
 
Epoch: 6/100 | Best Valid Score Until Now: 0.747 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.141 | Valid Score: 0.750
 
Epoch: 7/100 | Best Valid Score Until Now: 0.750 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.140 | Valid Score: 0.754
 
Epoch: 8/100 | Best Valid Score Until Now: 

# GCN 2Layer

In [18]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = GraphConv(self.node_feature_size, self.hidden_size, allow_zero_in_degree=True)
        self.conv2 = GraphConv(self.hidden_size, self.num_tasks, allow_zero_in_degree=True)

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [19]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

In [20]:
def loss_func(output, label, mask, num_tasks):
    #This tensor is used as a weight for positive examples in the binary cross-entropy loss.
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    #reduction='none': This specifies that the loss should not be reduced and returned element-wise.
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    #The mask tensor is used to mask out the loss for invalid labels.
    loss = mask*criterion(output,label)
    #The masked loss is summed and divided by the sum of the mask to compute the average loss.
    loss = loss.sum() / mask.sum()
    return loss

In [21]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        # Zeros out the gradients of the model parameters using
        optimizer.zero_grad(set_to_none=True)
        # Computes the gradients of the loss with respect to the model parameters using
        loss_train.backward()
        # Updates the model parameters using
        optimizer.step()
        # Adds the loss for the current batch to
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [22]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [23]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [24]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.463 | Valid Score: 0.498
 
Epoch: 1/100 | Best Valid Score Until Now: 0.498 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.168 | Valid Score: 0.508
 
Epoch: 2/100 | Best Valid Score Until Now: 0.508 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.164 | Valid Score: 0.520
 
Epoch: 3/100 | Best Valid Score Until Now: 0.520 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.162 | Valid Score: 0.534
 
Epoch: 4/100 | Best Valid Score Until Now: 0.534 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.160 | Valid Score: 0.554
 
Epoch: 5/100 | Best Valid Score Until Now: 0.554 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.158 | Valid Score: 0.577
 
Epoch: 6/100 | Best Valid Score Until Now: 0.577 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.158 | Valid Score: 0.606
 
Epoch: 7/100 | Best Valid Score Until Now: 0.606 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.154 | Valid Score: 0.635
 
Epoch: 8/100 | Best Valid Score Until Now: 

# GraphSage 3Layer

In [25]:
from dgl.nn import SAGEConv

In [26]:
class GNN(nn.Module):
    def __init__(self, config, global_size=200, num_tasks=1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size,aggregator_type='mean')
        self.conv2 = SAGEConv(self.hidden_size, self.hidden_size,aggregator_type='mean')
        self.conv3 = SAGEConv(self.hidden_size, self.num_tasks,aggregator_type='mean')

    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"] = mol_dgl_graph.ndata["v"][:, :self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:, :self.edge_feature_size]

        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        h = F.relu(h)
        h = self.conv3(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h

        return dgl.mean_nodes(mol_dgl_graph, "h")


In [27]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

In [28]:
def loss_func(output, label, mask, num_tasks):
    #This tensor is used as a weight for positive examples in the binary cross-entropy loss.
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    #reduction='none': This specifies that the loss should not be reduced and returned element-wise.
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    #The mask tensor is used to mask out the loss for invalid labels.
    loss = mask*criterion(output,label)
    #The masked loss is summed and divided by the sum of the mask to compute the average loss.
    loss = loss.sum() / mask.sum()
    return loss

In [29]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        # Zeros out the gradients of the model parameters using
        optimizer.zero_grad(set_to_none=True)
        # Computes the gradients of the loss with respect to the model parameters using
        loss_train.backward()
        # Updates the model parameters using
        optimizer.step()
        # Adds the loss for the current batch to
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [30]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [31]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [32]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.156 | Valid Score: 0.709
 
Epoch: 1/100 | Best Valid Score Until Now: 0.709 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.142 | Valid Score: 0.753
 
Epoch: 2/100 | Best Valid Score Until Now: 0.753 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.139 | Valid Score: 0.769
 
Epoch: 3/100 | Best Valid Score Until Now: 0.769 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.137 | Valid Score: 0.778
 
Epoch: 4/100 | Best Valid Score Until Now: 0.778 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.137 | Valid Score: 0.787
 
Epoch: 5/100 | Best Valid Score Until Now: 0.787 

Patience 1
Epoch: 6/100 | Training Loss: 0.134 | Valid Score: 0.786
 
Epoch: 6/100 | Best Valid Score Until Now: 0.787 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.133 | Valid Score: 0.792
 
Epoch: 7/100 | Best Valid Score Until Now: 0.792 

Patience 1
Epoch: 8/100 | Training Loss: 0.132 | Valid Score: 0.791
 
Epoch: 8/100 | Best Valid Score Until Now: 0.792 

Sa

# GraphSage 2 Layer

In [33]:
class GNN(nn.Module):
    def __init__(self, config, global_size = 200, num_tasks = 1):
        super().__init__()
        self.config = config
        self.num_tasks = num_tasks

        # Node feature size
        self.node_feature_size = self.config.get('node_feature_size', 127)

        # Edge feature size
        self.edge_feature_size = self.config.get('edge_feature_size', 12)

        # Hidden size
        self.hidden_size = self.config.get('hidden_size', 100)

        self.conv1 = SAGEConv(self.node_feature_size, self.hidden_size,aggregator_type='mean')
        self.conv2 = SAGEConv(self.hidden_size, self.num_tasks,aggregator_type='mean')

    # def forward(self, g, in_feat):
    def forward(self, mol_dgl_graph, globals):
        mol_dgl_graph.ndata["v"]= mol_dgl_graph.ndata["v"][:,:self.node_feature_size]
        mol_dgl_graph.edata["e"] = mol_dgl_graph.edata["e"][:,:self.edge_feature_size]
        h = self.conv1(mol_dgl_graph, mol_dgl_graph.ndata["v"])
        h = F.relu(h)
        h = self.conv2(mol_dgl_graph, h)
        mol_dgl_graph.ndata["h"] = h
        return dgl.mean_nodes(mol_dgl_graph, "h")

In [34]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, val_size, num_tasks):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0)
        labels_all= torch.empty(0)
        masks_all= torch.empty(0)
        for i, (mol_dgl_graph, labels, masks, globals) in enumerate(data_loader):
            prediction = model(mol_dgl_graph, globals)
            prediction = torch.sigmoid(prediction)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)
            masks_all = torch.cat((masks_all, masks), 0)
        average = torch.tensor([0.])
        for i in range(num_tasks):
            a1 = prediction_all[:, i][masks_all[:,i]==1]
            a2 = labels_all[:, i][masks_all[:,i]==1]
            try:
                t = metric(a2.int().cpu(), a1.cpu()).item()
            except ValueError:
                t = 0
            average += t
    return average.item()/num_tasks

In [35]:
def loss_func(output, label, mask, num_tasks):
    #This tensor is used as a weight for positive examples in the binary cross-entropy loss.
    pos_weight = torch.ones((1, num_tasks))
    pos_weight
    #reduction='none': This specifies that the loss should not be reduced and returned element-wise.
    criterion = torch.nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)
    #The mask tensor is used to mask out the loss for invalid labels.
    loss = mask*criterion(output,label)
    #The masked loss is summed and divided by the sum of the mask to compute the average loss.
    loss = loss.sum() / mask.sum()
    return loss

In [36]:
def train_epoch(train_dataloader, model, optimizer):
    epoch_train_loss = 0
    iterations = 0
    model.train() # Prepare model for training
    for i, (mol_dgl_graph, labels, masks, globals) in enumerate(train_dataloader):
        prediction = model(mol_dgl_graph, globals)
        loss_train = loss_func(prediction, labels, masks, num_tasks)
        # Zeros out the gradients of the model parameters using
        optimizer.zero_grad(set_to_none=True)
        # Computes the gradients of the loss with respect to the model parameters using
        loss_train.backward()
        # Updates the model parameters using
        optimizer.step()
        # Adds the loss for the current batch to
        epoch_train_loss += loss_train.detach().item()
        iterations += 1
    epoch_train_loss /= iterations
    return epoch_train_loss

In [37]:
def train_evaluate():

    model = GNN(config, global_size, num_tasks)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

    best_val = 0
    patience_count = 1
    epoch = 1

    while epoch <= num_epochs:
        if patience_count <= patience:
            model.train()
            loss_train = train_epoch(train_dataloader, model, optimizer)
            model.eval()
            score_val = compute_score(model, val_dataloader, len(val_set), num_tasks)
            if score_val > best_val:
                best_val = score_val
                print("Save checkpoint")
                path = os.path.join(checkpoint_path, 'checkpoint.pth')
                dict_checkpoint = {"score_val": score_val}
                dict_checkpoint.update({"model_state_dict": model.state_dict(), "optimizer_state": optimizer.state_dict()})
                with open(path, "wb") as outputfile:
                    cloudpickle.dump(dict_checkpoint, outputfile)
                patience_count = 1
            else:
                print("Patience", patience_count)
                patience_count += 1

            print("Epoch: {}/{} | Training Loss: {:.3f} | Valid Score: {:.3f}".format(
            epoch, num_epochs, loss_train, score_val))

            print(" ")
            print("Epoch: {}/{} | Best Valid Score Until Now: {:.3f}".format(epoch, num_epochs, best_val), "\n")
        epoch += 1

    # best model save
    shutil.rmtree(best_model_path, ignore_errors=True)
    shutil.copytree(checkpoint_path, best_model_path)

    print("Final results:")
    print("Average Valid Score: {:.3f}".format(np.mean(best_val)), "\n")


In [38]:
def test_evaluate():
    final_model = GNN(config, global_size, num_tasks)
    path = os.path.join(best_model_path, 'checkpoint.pth')
    with open(path, 'rb') as f:
        checkpoint = cloudpickle.load(f)
    final_model.load_state_dict(checkpoint["model_state_dict"])
    final_model.eval()
    test_score = compute_score(final_model, test_dataloader, len(test_set), num_tasks)

    print("Test Score: {:.3f}".format(test_score), "\n")
    print("Execution time: {:.3f} seconds".format(time.time() - start_time))


In [39]:
import time
start_time = time.time()

train_evaluate()
test_evaluate()

Save checkpoint
Epoch: 1/100 | Training Loss: 0.332 | Valid Score: 0.498
 
Epoch: 1/100 | Best Valid Score Until Now: 0.498 

Save checkpoint
Epoch: 2/100 | Training Loss: 0.163 | Valid Score: 0.520
 
Epoch: 2/100 | Best Valid Score Until Now: 0.520 

Save checkpoint
Epoch: 3/100 | Training Loss: 0.156 | Valid Score: 0.590
 
Epoch: 3/100 | Best Valid Score Until Now: 0.590 

Save checkpoint
Epoch: 4/100 | Training Loss: 0.150 | Valid Score: 0.652
 
Epoch: 4/100 | Best Valid Score Until Now: 0.652 

Save checkpoint
Epoch: 5/100 | Training Loss: 0.147 | Valid Score: 0.695
 
Epoch: 5/100 | Best Valid Score Until Now: 0.695 

Save checkpoint
Epoch: 6/100 | Training Loss: 0.144 | Valid Score: 0.724
 
Epoch: 6/100 | Best Valid Score Until Now: 0.724 

Save checkpoint
Epoch: 7/100 | Training Loss: 0.144 | Valid Score: 0.739
 
Epoch: 7/100 | Best Valid Score Until Now: 0.739 

Save checkpoint
Epoch: 8/100 | Training Loss: 0.141 | Valid Score: 0.749
 
Epoch: 8/100 | Best Valid Score Until Now: 